In [1]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
np.random.seed(0)

from sklearn.model_selection import train_test_split

from sklearn.mixture import GaussianMixture
from sklearn.cluster import KMeans
from sklearn.neighbors import KernelDensity

In [ ]:
num_clusters = 3

gmm = GaussianMixture(n_components=num_clusters, covariance_type='full')
gmm.fit(data)

plt.scatter(data[:,0], data[:,1])
plt.xlabel('$x_1$')
plt.ylabel('$x_2$')
plt.scatter(gmm.means_[:,0],gmm.means_[:,1])
plt.scatter(means[:,0],means[:,1])
plt.legend(['Data','Estimated mean','True mean'])
plt.show()

print('weights:\n {}\n'.format(gmm.weights_))
print('means:\n {}\n'.format(gmm.means_))
print('covariances:\n {}\n'.format(gmm.covariances_))